In [1]:
import numpy as np
import pyfits
import math
from math import log10
import matplotlib.pyplot as plt
import scipy
from scipy.stats import binned_statistic
from scipy.stats import binned_statistic_2d
from matplotlib import rc
import pickle
import os
import astropy as ap
from astropy.cosmology import WMAP7
from scipy import interpolate
import matplotlib.patheffects as patheffects
from matplotlib import rcParams
from matplotlib.path import Path
import matplotlib.patches as patches
import matplotlib as mpl
from matplotlib import rc
from matplotlib import colors
from __future__ import unicode_literals

rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["Computer Modern Sans"]
rcParams["text.usetex"] = True
rcParams["text.latex.preamble"] = r"\usepackage{cmbright}"
rcParams['xtick.major.pad']='10'
rcParams['ytick.major.pad']='10'
rcParams['font.size'] = 20
rcParams['axes.labelsize'] = 17
rcParams['axes.labelweight'] = 'bold'
rcParams['axes.titlesize'] = 16
rcParams['xtick.labelsize'] = 13
rcParams['ytick.labelsize'] = 13
rcParams['legend.fontsize'] = 15
rcParams['figure.titlesize'] = 1

/usr/local/lib/python3.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
os.chdir('/Users/nitya/Work/Data/Project1/measurements')

file2 = open(r'SFRP.pkl', 'rb')
d = pickle.load(file2, encoding = 'latin')
file2.close()
print(np.shape(d))

# There's one annoying 0 entry in N band nmgy_ivar - removing it!
indf = np.where(np.asarray(d['NMGY_IVAR'])[:,1]>0)[0]
d = d[indf]

ind, z, mtol, kc = d['NSAID'], d['Z'], d['MTOL'], d['KCORRECT']
opt = [d['ABSMAG'][:,1][i]-d['ABSMAG'][:,4][i] for i in range(len(ind))]
inf = [d['W1MAG'][i]-d['W3MAG'][i] for i in range(len(ind))]

(75476,)


In [3]:
class nmgy_to_jy:
    
    def __init__(self):
        self.flux_nmgy = flux_nmgy
        #kc is kcorrect
        self.kc = kc
        self.ivar_nmgy = ivar_nmgy
    
    #incorporating k-correct;
    def jansky(flux_nmgy, kc):
        flux_jy = flux_nmgy*3631*(10.0**(-9.0))*(10**(kc/(-2.5)))
        return flux_jy
    def jansky_err(ivar_nmgy, kc):
        flux_jy_err = (ivar_nmgy**(-0.5))*3631*(10.0**(-9.0))*(10**(kc/(-2.5)))
        return flux_jy_err
    
    #just flux to Jy conversion without k-corrects;
    def jansky0(flux_nmgy):
        flux_jy = flux_nmgy*3631*(10.0**(-9.0))
        return flux_jy
    def jansky_err0(ivar_nmgy):
        flux_jy_err = (ivar_nmgy**(-0.5))*3631*(10.0**(-9.0))
        return flux_jy_err

In [4]:

#optical fluxes in nanomaggies
opt_flux = d['NMGY']
opt_flerr = d['NMGY_IVAR']
#WISE fluxes in nanomaggies
inf_flux = np.column_stack((d['W1_NMGY'],d['W2_NMGY'],d['W3_NMGY'],d['W4_NMGY']))
inf_flerr = np.column_stack((d['W1SIGM_NMGY'],d['W2SIGM_NMGY'],d['W3SIGM_NMGY'],d['W4SIGM_NMGY']))

flux = nmgy_to_jy.jansky(opt_flux,kc)
flux_err = nmgy_to_jy.jansky_err(opt_flerr,kc)

#Original iteration
wflux = nmgy_to_jy.jansky0(inf_flux)
wflux_err = nmgy_to_jy.jansky_err0(inf_flerr)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in power


In [5]:
#If we're talking about a different iteration:

# file1 = open(r'WISE_kcorrects_a', 'rb')
# w = pickle.load(file1, encoding = 'latin')
# file1.close()
# print(w.dtype.names)
# print(np.shape(w))

# wkc = np.column_stack((w['kc1'], w['kc2'], w['kc3'], w['kc4']))
# wflux = nmgy_to_jy.jansky(inf_flux,wkc)
# w1fluxerr = nmgy_to_jy.jansky_err(inf_flerr,wkc)

In [6]:
# Magphys input file format;
a = np.column_stack((flux[:,0], flux_err[:,0], flux[:,1], flux_err[:,1], flux[:,2], flux_err[:,2], 
                     flux[:,3], flux_err[:,3], flux[:,4], flux_err[:,4], flux[:,5], flux_err[:,5], 
                     flux[:,6], flux_err[:,6], wflux[:,0], wflux_err[:,0], wflux[:,1], 
                     wflux_err[:,1], wflux[:,2], wflux_err[:,2], wflux[:,3], wflux_err[:,3]))

# Setting r-band flux to 1
r_flux = a[:,8]
new_a = np.asarray([[a[i][j]/r_flux[i] for j in range(22)] for i in range(len(a))])

In [7]:
#Binning in the color-color space and averaging the fluxes in each bin
h = binned_statistic_2d(inf,opt,ind,statistic = 'count',bins = (25,25))
b = np.reshape(np.asarray(h[3]),(len(ind),1))
f = np.append(new_a,b, axis = 1)
f = f[f[:,22].argsort()]
bins = np.unique(f[:,22])

n = [np.where(f[:,22] == bins[i]) for i in range(len(bins))]
data = np.array([np.mean(f[n[i]], axis = 0) for i in range(len(bins))])

/usr/local/lib/python3.7/site-packages/scipy/stats/_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


In [10]:
redshifts = np.zeros(len(bins))
stuff = np.column_stack((bins,redshifts,data[:,0:22]))

# filename = 'magphys_input_file_kc1'
# f = open(filename,'w')
# for line in stuff:
#     f.write("  ".join(str(x) for x in line) + "\n")
# f.close()